## qflib Library Examples

In [ ]:
import qflib as qf
import numpy as np
import os
import matplotlib.pyplot as plt
plt.style.use('ggplot')

print("qflib version: {0}".format(qf.version()))
print("pid: {0}".format(os.getpid()))

In [ ]:
name = "World"
print(qf.sayHello(name))

In [ ]:
x = [1, 2, 3]
y = [4, 5]
op = qf.outerProd(x, y)
print(f'x: {x}\ny: {y}')
print(f'outerProd:\n{op}')

In [ ]:
# Matrix Echo
m = np.array([[1, 2, 3], [4, 5, 6]])
em = qf.echoMatrix(m)
print(f'orig matrix:\n{m}')
print(f'echo matrix:\n{em}')

In [ ]:
# Piecewise Polynomial Curves
xbpt = np.arange(1, 6)
yval =  np.arange(-10, 15, 5)
pord = 1
xval = np.arange(0.5, 5.55, 0.05)
pval = qf.ppolyEval(xbpt, yval, pord, xval, 0)
pder = qf.ppolyEval(xbpt, yval, pord, xval, 1)
pint = qf.ppolyIntegral(xbpt, yval, pord, xval[0], xval)

# plot
plt.plot(xbpt, yval, color='black', marker='o')
plt.plot(xval, pval, color='blue', label="values")
plt.plot(xval, pder, color='green', label="slopes")
plt.plot(xval, pint, color='red', label="integrals")
plt.title("Piecewise Polynomial Curve")
plt.legend(loc="upper left")
plt.xlabel('x')
plt.ylabel('y');

In [ ]:
#Root bracketing and secant method search

pcoeffs = [0, 2, 1]
lolim = -4
uplim = 3
nsubs = 10

brkts = qf.polyBracket(pcoeffs, lolim, uplim, nsubs)
print(f"Roots of polynomial: {pcoeffs}")
print(f"Root brackets:\n {brkts}")

root = qf.polySecant(pcoeffs, brkts[0, 0], brkts[0, 1], 1e-12)
print(f"root: {root:.4f}")

In [ ]:
#ToContCmpd, FromContComp

anfreq = 1
inrate = 0.10
ccrate = qf.toContCmpd(inrate, anfreq)
outrate = qf.fromContCmpd(ccrate, anfreq)
print('To and from continuous compounding')
print(f'InRate={inrate:.4f}, CCRate={ccrate:.4f}, PerRate={outrate:.4f}')

### Analytic Prices

In [ ]:
#fwdprice
fwdpx = qf.fwdPrice(spot = 100, timetoexp = 1.0, intrate = 0.02, divyield = 0.04)
print('Forward price analytic solution')
print(f'Price={fwdpx:.4f}')

In [ ]:
#digibs
digi = qf.digiBS(payofftype = 1, spot = 100, timetoexp = 1.0, strike = 100,
                  intrate = 0.02, divyield = 0.04, volatility = 0.2)[0]
print('Digital option using Black-Scholes analytic solution')
print(f'Price={digi:.4f}')

In [ ]:
#eurobs
euro = qf.euroBS(payofftype = 1, spot = 100, timetoexp = 1.0, strike = 100,
                  intrate = 0.02, divyield = 0.04, volatility = 0.4)[0]
print('European option using Black-Scholes analytic solution')
print(f'Price={euro:.4f}')

### Market Objects

In [ ]:
#yccreate
ycname = "USD"
yc = qf.ycCreate(ycname = ycname, 
                  tmats = [1/12,   1/4,  1/2,   3/4,    1,     2,    3,     4,    5,      10], 
                  vals =  [0.01,  0.02, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.0575, 0.065],
                  valtype = 0)
print(f'Created yield curve: {yc}')

In [ ]:
#discount, spotrate, fwdrate
tmat = 2.0 # years
tfix = 1.0 # years
df = qf.discount(ycname = yc, tmat = tmat)
spotrate = qf.spotRate(ycname = yc, tmat = tmat)
fwdrate = qf.fwdRate(ycname = yc, tmat1 = tfix, tmat2 = tmat)
print(f'Maturity={tmat} years, DF={df:.4f}, SpotRate={spotrate:.4f}, FwdRate={fwdrate:.4f}')

In [ ]:
#create a 2 year weekly array
tweekly = [i / 52.0 for i in range(1, 105)]
spotRates = [qf.spotRate(yc, tmat) for tmat in tweekly] 
fwdRates = [qf.fwdRate(yc, tmat1, tmat2 ) for tmat1, tmat2 in zip(tweekly[:-1], tweekly[1:])]
fwdRates.insert(0, qf.fwdRate(yc, 0, tweekly[0]))

In [ ]:
# plot
plt.plot(tweekly, spotRates, color='black', label="spot rate")
plt.plot(tweekly, fwdRates, color='blue', label="fwd rate")
plt.title("Spot and Forward Rates")
plt.legend(loc="upper left")
plt.xlabel('maturity (yr)')
plt.ylabel('rate');

In [ ]:
# list market contents
print(qf.mktList())

### Monte Carlo

In [ ]:
# European Option

mcpars0 = {'URNGTYPE': 'MT19937', 'PATHGENTYPE': 'EULER'}
npaths0 = 200000
euromc0 = qf.euroBSMC(payofftype = 1, strike = 100, timetoexp = 1.0, spot = 100,
                       discountcrv =  yc, divyield = 0.02, volatility = 0.4,
                       mcparams = mcpars0, npaths = npaths0)
print(f'URNGTYPE={mcpars0["URNGTYPE"]} PATHGENTYPE={mcpars0["PATHGENTYPE"]} NPATHS={npaths0}')
print(f'Price={euromc0['Mean']:0.4f}  StdErr={euromc0['StdErr']:0.4f}')

In [ ]:
# convergence analysis
npaths = [ 100 * 2 ** i for i in range(12)]

mcprices = []
stderrs = []
for i in range(len(npaths)):
    mcres = qf.euroBSMC(payofftype = 1, strike = 100, timetoexp = 1.0, spot = 100,
                       discountcrv =  yc, divyield = 0.02, volatility = 0.4,
                       mcparams = mcpars0, npaths = npaths[i])
    mcprices.append(mcres["Mean"])
    stderrs.append(mcres["StdErr"])

bsprice = qf.euroBS(payofftype = 1, spot = 100, timetoexp = 1.0, strike = 100,
                  intrate = 0.04, divyield = 0.02, volatility = 0.4)[0]

# plot prices and errors
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(npaths, mcprices, color='black', marker='o')
plt.axhline(y=bsprice, color='blue', linestyle='--')
plt.xscale('log')
plt.title("MC Price")
plt.xlabel('Num Paths')
plt.ylabel('Price')

plt.subplot(1, 2, 2)
plt.plot(npaths, stderrs, color='black', marker='o')
#plt.axhline(y=bsprice, color='blue', linestyle='--')
plt.xscale('log')
plt.title("MC Error")
plt.xlabel('Num Paths')
plt.ylabel('Std Error');


In [ ]:
# Asian Basket Option

fixtimes = [  0.5,   1.0,   1.5,   2.0]
assqts   = [  0.2,   0.2,   0.2,   0.2,   0.2]
spots    = [100.0, 100.0, 100.0, 100.0, 100.0]
divylds  = [ 0.02,  0.02,  0.02,  0.02,  0.02]
vols     = [ 0.30,  0.30,  0.30,  0.30,  0.30]
correls  = np.array([
     [1.0,  0.5,  0.7, 0.6, 0.6],
     [0.5,  1.0,  0.8, 0.7, 0.5],
     [0.7,  0.8,  1.0, 0.7, 0.5], 
     [0.6,  0.7,  0.7, 1.0, 0.6],
     [0.6,  0.5,  0.5, 0.6, 1.0]])

mcpars1 = {'URNGTYPE': 'MT19937', 'PATHGENTYPE': 'EULER'}
npaths1 = 200000
asianmc = qf.asianBasketBSMC(payofftype = 1, strike = 100, fixtimes = fixtimes, 
                             assetquantities = assqts, spots = spots,
                             discountcrv =  yc, divyields = divylds, 
                             volatilities= vols, correlmat = correls,
                             mcparams = mcpars1, npaths = npaths1)
print(f'URNGTYPE={mcpars1["URNGTYPE"]} PATHGENTYPE={mcpars1["PATHGENTYPE"]} NPATHS={npaths1}')
print(f'Price={asianmc['Mean']:0.4f}  StdErr={asianmc['StdErr']:0.4f}')


### PDE

In [ ]:
# European option

pdepars0 = {'NTIMESTEPS': 50, 'NSPOTNODES': 50, 'NSTDDEVS': 4, 'THETA': 0}

europde0 = qf.euroBSPDE(payofftype = 1, strike = 100, timetoexp = 1.0, spot = 100,
                        discountcrv =  yc, divyield = 0.02, volatility = 0.4, pdeparams = pdepars0)

print(f'NTIMESTEPS={pdepars0["NTIMESTEPS"]} NSPOTNODES={pdepars0["NSPOTNODES"]} NSTDDEVS={pdepars0["NSTDDEVS"]} THETA={pdepars0["THETA"]}')
print(f'Price={europde0["Price"]:0.4f}')

pdepars1 = {'NTIMESTEPS': 50, 'NSPOTNODES': 50, 'NSTDDEVS': 4, 'THETA': 1}

europde1 = qf.euroBSPDE(payofftype = 1, strike = 100, timetoexp = 1.0, spot = 100,
                         discountcrv =  yc, divyield = 0.02, volatility = 0.4, pdeparams = pdepars1)

print(f'NTIMESTEPS={pdepars1["NTIMESTEPS"]} NSPOTNODES={pdepars1["NSPOTNODES"]} NSTDDEVS={pdepars1["NSTDDEVS"]} THETA={pdepars1["THETA"]}')
print(f'Price={europde1["Price"]:0.4f}')

pdepars2 = {'NTIMESTEPS': 50, 'NSPOTNODES': 50, 'NSTDDEVS': 4, 'THETA': 0.5}

europde2 = qf.euroBSPDE(payofftype = 1, strike = 100, timetoexp = 1.0, spot = 100,
                         discountcrv =  yc, divyield = 0.02, volatility = 0.4, pdeparams = pdepars2)

print(f'NTIMESTEPS={pdepars2["NTIMESTEPS"]} NSPOTNODES={pdepars1["NSPOTNODES"]} NSTDDEVS={pdepars2["NSTDDEVS"]} THETA={pdepars2["THETA"]}')
print(f'Price={europde2["Price"]:0.4f}')

# exact price
eurobs = qf.euroBS(payofftype = 1, strike = 100, timetoexp = 1.0, spot = 100,
                   intrate = qf.spotRate(yc, 1.0), divyield = 0.02, volatility = 0.4)[0]

print(f'BSPrice={eurobs:0.4f}')


In [ ]:
# convergence analysis

refines = [i for i in range(0, 6)]
euroerrs = []
pdepars = {'NTIMESTEPS': 100, 'NSPOTNODES': 100, 'NSTDDEVS': 4, 'THETA': 1}

for i in refines:
    europx = qf.euroBSPDE(payofftype = 1, strike = 100, timetoexp = 1.0, spot = 100,
                          discountcrv = yc, divyield = 0.02, volatility = 0.4, pdeparams = pdepars)
    euroerrs.append(europx['Price'] - eurobs)
    pdepars['NTIMESTEPS'] = 2 * pdepars['NTIMESTEPS']
    pdepars['NSPOTNODES'] = 2 * pdepars['NSPOTNODES']

# quadratic and linear errors
quaderrs = [euroerrs[0]]
linerrs = [euroerrs[0]]

for i in refines[1:]:
    quaderrs.append(quaderrs[i - 1] / 4)
    linerrs.append(0.0001 + linerrs[i - 1] / 2)


In [ ]:
# plot errors
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(refines, euroerrs, color='red', marker='o', label='PDE Error')
plt.plot(refines, quaderrs, color='blue', marker='o', label='Quadratic')
plt.plot(refines, linerrs, color='black', marker='o', label='Linear')
plt.title("European PDE Error Scaling")
plt.xlabel('Refinement')
plt.ylabel('Error')
plt.legend()